In [4]:
#Bibliotecas
!pip install yahooquery
!pip install pandas_ta

import yfinance
import matplotlib.pyplot as plt
import math
from prophet import Prophet
from yahooquery import Ticker
import pandas as pd
from datetime import datetime
import pandas_ta as ta
import plotly.express as px
import plotly.graph_objs as go
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=a3bebff35242400d62aad25eb81213aa9f58b6977fde70d5e5d653181d4ee57d
  Stored in directory: /root/.cache/pip/wheels/1e/5c/47/759b32beb377aee150eb4c1f1b2b7635032cc19ea09ce076fd
Successfully built pandas_ta


In [6]:
# Nome na Bolsa de Valores.
symbol = "WEGE3.SA"

# Consulta de Estoque
stock = Ticker(symbol)

# Pegar 4 anos de intervalo
history = stock.history(period="48mo")

# Deixar somente a data
history.reset_index(level=["symbol"], inplace=True)
history['date'] = history.index

history.set_index(pd.DatetimeIndex(history.index), inplace=True)

# selecionar features de interesse
data = history[['date','adjclose']].copy();

# Analisar 21 dias e anexar ao conjunto de dados
data.ta.ema(close='adjclose', length=21, append=True)

# Negar valores nulos
data.dropna(inplace=True)

In [7]:
#Plotagem
fig = px.line(data, x='date', y='adjclose')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

In [8]:
#DIVISÅO TREINO-TESTE
# Modelo de treino
df_train = data[['date','adjclose','EMA_21']]

df_train = df_train.rename(columns={"date": "ds", "adjclose": "y"})

# Fit model
m = Prophet(daily_seasonality=True)
m.fit(df_train)

# Prever 30 dias de açoes
future = m.make_future_dataframe(periods=30)

# Somente dias úteis
future['day'] =  future['ds'].dt.weekday
future = future[future['day'] <=4]

# Data de prediçao
forecast = m.predict(future)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpovnkfxza/4b2jer99.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpovnkfxza/5pvf4_5v.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=40862', 'data', 'file=/tmp/tmpovnkfxza/4b2jer99.json', 'init=/tmp/tmpovnkfxza/5pvf4_5v.json', 'output', 'file=/tmp/tmpovnkfxza/prophet_model336vix_0/prophet_model-20230409124218.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:42:18 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:42:18 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [9]:
# Plotagem do resultado
fig = go.Figure([
    go.Scatter(x=df_train['ds'], y=df_train['y'], name='Actual', mode='lines'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted', mode='lines'),
    go.Scatter(x=forecast['ds'], y=df_train['EMA_21'], name='EMA', mode='lines')
])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [10]:
# Pegar todas as prediçoes
pred_df = forecast[forecast['ds'] > datetime.today()][['ds','yhat']]

# Resetar Index
pred_df.reset_index(inplace=True)
pred_df.drop(labels='index', axis=1,inplace=True)
pred_df.rename(columns={'ds': 'date', 'yhat': 'predicted price'}, inplace=True)
pred_df

,date,predicted price
0,2023-04-10,40.579922
1,2023-04-11,40.674189
2,2023-04-12,40.864002
3,2023-04-13,40.906213
4,2023-04-14,40.900074
5,2023-04-17,40.628257
6,2023-04-18,40.592983
7,2023-04-19,40.652012
8,2023-04-20,40.564138
9,2023-04-21,40.430545


In [11]:
# Executar validaçao

df_cv = cross_validation(m, initial='720 days', period='30 days', horizon = '365 days')

# visualize data
print(df_cv.head())

# measure performance
df_p = performance_metrics(df_cv)
print(df_p.head())

INFO:prophet:Making 12 forecasts with cutoffs between 2021-05-11 00:00:00 and 2022-04-06 00:00:00


  0%|          | 0/12 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpovnkfxza/7fqh7n3a.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpovnkfxza/x8nbf9jo.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=19483', 'data', 'file=/tmp/tmpovnkfxza/7fqh7n3a.json', 'init=/tmp/tmpovnkfxza/x8nbf9jo.json', 'output', 'file=/tmp/tmpovnkfxza/prophet_modelux4_8fic/prophet_model-20230409124235.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:42:35 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:42:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpovnkfxza/vzacn6a5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpovnkfxza/4_clpk02.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

          ds       yhat  yhat_lower  yhat_upper          y     cutoff
0 2021-05-12  33.493916   31.743565   35.293078  30.308985 2021-05-11
1 2021-05-13  33.494064   31.798449   35.245816  30.822693 2021-05-11
2 2021-05-14  33.422648   31.759961   35.191136  31.113472 2021-05-11
3 2021-05-17  33.315762   31.636660   35.033869  31.181320 2021-05-11
4 2021-05-18  33.458082   31.806340   35.190453  31.404259 2021-05-11
  horizon        mse      rmse       mae      mape     mdape     smape  \
0 37 days  20.362053  4.512433  3.675945  0.111769  0.088407  0.114734   
1 38 days  20.776753  4.558152  3.723342  0.113235  0.089056  0.116347   
2 39 days  21.235359  4.608184  3.781714  0.115014  0.092677  0.118060   
3 40 days  21.602980  4.647901  3.825097  0.116241  0.093244  0.119331   
4 41 days  21.961777  4.686339  3.867505  0.117444  0.094096  0.120606   

   coverage  
0  0.232963  
1  0.226667  
2  0.218333  
3  0.209048  
4  0.207500  
